# US hurricane #

1. **Data Loading & Cleaning:**
- The function `load_and_clean_data` reads the hurricane CSV file, removes rows with missing or anomalous values (including wind speeds >200 knots), and cleans the hurricane names.
- Feature engineering is performed by creating new columns such as `decade`, `name_initial` (only if `name_clean` is valid), `intensity_metric`, `strong_hurricane` (flagging Category 4 & 5), and both categorical and numeric versions of computed hurricane categories.

2. **Visualization Functions:**
- Separate functions create Plotly figures:
    - ***Scatter Plot:*** A bubble chart showing max wind speed vs. central pressure.
    - ***Strong Hurricanes Line Plot:*** Displays the count of strong hurricanes per decade with a moving average, linear regression, and polynomial regression.
    - ***Choropleth Map:*** Shows hurricane frequency by state, using the exploded `states_list` column.
    - ***Category Bar Chart & Box Plot:*** Visualize hurricane counts by category and wind speed distributions.
    - ***Histogram:*** Displays the distribution of max wind speeds.
    - ***Stacked Bar Chart:*** Shows hurricane name initials by decade, with an optional top‑N filter.

3. **App Layout:**
- The layout uses Dash Bootstrap Components and is organized into rows and tabs.
- Filters (dropdowns, slider, and input) are provided at the top.
- Tabs (Overview, Geography, Categories, Naming Trends, Distributions) display corresponding graphs and analysis.
- By default, the year range slider is set to cover the entire dataset period.

4. **Callbacks:**
- The main callback updates all figures and analysis texts based on the selected filters, including additional trend analyses for strong hurricanes and naming trends (using exponential smoothing).
- A separate callback resets all filters to default values when the reset button is clicked.

5. **Running the App:**
- Finally, the Dash server is launched in debug mode.

In [2]:
import re
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output, State, no_update
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import linregress, chi2_contingency

### 1. Data Loading, Cleaning, and Feature Engineering Function ###

In [4]:
def load_and_clean_data():
    """
    Loads data from a CSV file,
    cleans the data (removes extra quotes, processes categories, extracts state codes),
    handles missing values and anomalies (e.g. negative wind speeds, extreme wind speeds > 200 knots),
    and performs feature engineering.
    
    In addition, it computes:
      - 'computed_category' based on max wind speed (in knots) using the Saffir-Simpson scale:
          <34: Tropical Depression
          34–63: Tropical Storm
          64–82: Category 1
          83–95: Category 2
          96–112: Category 3
          113–136: Category 4
          ≥137: Category 5
      - 'computed_numeric_category': numeric representation (0 for depression/storm, then 1–5 for categories)
    
    :param url: CSV file.
    :return: Cleaned and enriched DataFrame.
    """
    df = pd.read_csv('us-hurricanes.csv')

    # Remove rows with missing critical numeric data or anomalies
    df = df.dropna(subset=['max-wind-(kt)', 'central-pressure-(mb)', 'year'])
    df = df[(df['max-wind-(kt)'] >= 0) & (df['central-pressure-(mb)'] > 0)]

    # Filter out extreme wind speed outliers (>200 knots)
    df = df[df['max-wind-(kt)'] <= 200]

    # Clean hurricane names by removing extra quotes
    df['name_clean'] = df['name'].astype(str).str.replace('"', '', regex=False)

    # Process the 'category' column: replace 'TS' (tropical storm) with NaN and convert to numeric
    df['category'] = df['category'].replace('TS', pd.NA)
    df['category'] = pd.to_numeric(df['category'],errors='coerce')

    # Process the states column: extract two-letter state codes using regex into 'states_list'
    df['states_list'] = df['states-affected-and-category-by-states'].apply(
        lambda x: re.findall(r'\b[A-Z]{2}\b', str(x))
    )

    # Feature Engineering:
    # a. Create a 'decade' column for grouping by decades
    df['decade'] = (df['year'] // 10) * 10

    # b. Extract the first letter of the cleaned hurricane names for naming trends analysis.
    #    Check if name_clean is not NaN and not empty; otherwise, assign NaN.
    df['name_initial'] = df['name_clean'].apply(lambda x: x[0] if pd.notnull(x) and len(x) > 0 else np.nan)
    
    # c. Calculate an example intensity metric (not meteorologically validated)
    df['intensity_metric'] = df['max-wind-(kt)'] / df['central-pressure-(mb)']

    # d. Flag strong hurricanes (Category 4 & 5)
    df['strong_hurricane'] = df['category'].apply(lambda x: 1 if x in [4, 5] else 0)

    # e. Compute 'computed_category' based on max wind speed thresholds
    def compute_category(wind):
        if wind < 34:
            return "Tropical Depression"
        elif wind < 64:
            return "Tropical Storm"
        elif wind <= 82:
            return "Category 1"
        elif wind <= 95:
            return "Category 2"
        elif wind <= 112:
            return "Category 3"
        elif wind <= 136:
            return "Category 4"
        else:
            return "Category 5"

    df['computed_category'] = df['max-wind-(kt)'].apply(compute_category)

    # f. Compute a numeric version for computed category for averaging intensity.
    def compute_numeric_category(wind):
        if wind < 34:
            return 0
        elif wind < 64:
            return 0  # Tropical Storm as 0
        elif wind <= 82:
            return 1
        elif wind <= 95:
            return 2
        elif wind <= 112:
            return 3
        elif wind <= 136:
            return 4
        else:
            return 5

    df['computed_numeric_category'] = df['max-wind-(kt)'].apply(compute_numeric_category)

    return df

### 2. Visualization Functions ###

In [6]:
def create_scatter_plot(df: pd.DataFrame) -> px.scatter:
    """
    Creates a bubble chart: max wind speed vs central pressure,
    with bubble size based on intensity metric.
    """
    fig = px.scatter(
        df,
        x='max-wind-(kt)',
        y='central-pressure-(mb)',
        color='category',
        size='intensity_metric',
        hover_data=['name_clean', 'year'],
        template='plotly_dark',
        title='Max Wind Speed vs Central Pressure (Bubble Chart)'
    )
    return fig

In [7]:
def create_strong_hurricanes_line_plot(df: pd.DataFrame) -> go.Figure:
    """
    Creates a line plot showing the count of Category 4 & 5 hurricanes per decade,
    augmented with a moving average, a linear regression line, and a polynomial (degree 2) regression line.
    """
    strong_df = df[df['strong_hurricane'] == 1]
    trend_df = strong_df.groupby('decade', as_index=False)['strong_hurricane'].sum()
    trend_df.rename(columns={'strong_hurricane': 'count'}, inplace=True)
    trend_df = trend_df.sort_values('decade')

    # Calculate moving average (window=2 decades)
    trend_df['moving_avg'] = trend_df['count'].rolling(window=2, min_periods=1).mean()

    # Linear regression
    if len(trend_df) >= 2:
        slope, intercept, r_value, p_value, std_err = linregress(trend_df['decade'], trend_df['count'])
        trend_df['linear_reg'] = intercept + slope * trend_df['decade']
    else:
        trend_df['linear_reg'] = trend_df['count']

    # Polynomial regression (degree 2)
    if len(trend_df) >= 3:
        poly_coef = np.polyfit(trend_df['decade'], trend_df['count'], 2)
        trend_df['poly_reg'] = np.polyval(poly_coef, trend_df['decade'])
    else:
        trend_df['poly_reg'] = trend_df['count']

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=trend_df['decade'],
        y=trend_df['count'],
        mode='lines+markers',
        name='Count'
    ))
    fig.add_trace(go.Scatter(
        x=trend_df['decade'],
        y=trend_df['moving_avg'],
        mode='lines',
        name='Moving Average'
    ))

    fig.add_trace(go.Scatter(
        x=trend_df['decade'],
        y=trend_df['linear_reg'],
        mode='lines',
        name='Linear Regression'
    ))

    fig.add_trace(go.Scatter(
        x=trend_df['decade'],
        y=trend_df['poly_reg'],
        mode='lines',
        name='Polynomial Regression (deg 2)'
    ))

    fig.update_layout(
        template='plotly_dark',
        title='Count of Category 4 & 5 Hurricanes per Decade',
        xaxis_title='Dacade',
        yaxis_title='Count'
    )

    return fig

In [8]:
def create_choropleth_map(df: pd.DataFrame) -> px.choropleth:
    """
    Creates a choropleth map of the USA showing the frequency of hurricanes by state.
    Hover data includes the hurricane count and sample hurricane names.
    """
    # Note: Use 'states_list' (with an "s") consistently.
    df_exploded = df.explode('states_list')
    state_counts = df_exploded.groupby('states_list').agg(
        count=('states_list', 'size'),
        example_names=('name_clean', lambda x: ", ".join(pd.unique(x)[:3]))
    ).reset_index()
    state_counts.rename(columns={'states_list': 'state'}, inplace=True)
    
    fig = px.choropleth(
        state_counts,
        locations='state',
        locationmode='USA-states',
        color='count',
        scope='usa',
        template='plotly_dark',
        title='Frequency of Hurricanes by State',
        hover_data={
            'count': True,
            'example_names': True
        }
    )
    return fig

In [9]:
def create_category_bar_chart(df: pd.DataFrame) -> px.bar:
    """
    Creates a bar chart showing the frequency of hurricanes by category.
    """
    cat_df = df.dropna(subset=['category'])
    cat_counts = cat_df['category'].value_counts().reset_index()
    cat_counts.columns = ['category', 'count']
    cat_counts.sort_values('category', inplace=True)
    fig = px.bar(
        cat_counts,
        x='category',
        y='count',
        template='plotly_dark',
        title='Frequency of Hurricanes by Category'
    )
    return fig

In [10]:
def create_box_plot(df: pd.DataFrame) -> px.box:
    """
    Creates a box plot showing the distribution of max wind speed for each hurricane category.
    """
    fig = px.box(
        df.dropna(subset=['category']),
        x='category',
        y='max-wind-(kt)',
        template='plotly_dark',
        title='Wind Speed Distribution by Category'
    )
    return fig

In [11]:
def create_histogram(df: pd.DataFrame) -> px.histogram:
    """
    Creates a histogram for the distribution of max wind speeds.
    """
    fig = px.histogram(
        df,
        x='max-wind-(kt)',
        template='plotly_dark',
        title='Distribution of Max Wind Speed'
    )
    return fig

In [12]:
def create_stacked_bar_chart(df: pd.DataFrame, top_n: int = None) -> px.bar:
    """
    Creates a stacked bar chart of hurricane name initials by decade.
    If top_n is provided, only the top N most frequent initials (overall) are displayed;
    otherwise, all initials are shown.
    """
    stacked_df = df.groupby(['decade', 'name_initial']).size().reset_index(name='count')
    if top_n is not None:
        overall_freq = stacked_df.groupby('name_initial')['count'].sum()
        top_initials = overall_freq.sort_values(ascending=False).head(top_n).index.tolist()
        stacked_df = stacked_df[stacked_df['name_initial'].isin(top_initials)]
    else:
        top_initials = sorted(stacked_df['name_initial'].unique())
    
    stacked_df = stacked_df.sort_values(['decade', 'count'], ascending=[True, False])
    fig = px.bar(
        stacked_df,
        x='decade',
        y='count',
        color='name_initial',
        barmode='stack',
        template='plotly_dark',
        title='Hurricane Name Initials by Decade',
        category_orders={'name_initial': top_initials}
    )
    return fig

### 3. Create Dash Application with Tabs and Enhanced UI/UX ###

In [14]:
external_stylesheets = [dbc.themes.DARKLY]
app = Dash(
    __name__,
    external_stylesheets=external_stylesheets
)

In [15]:
df = load_and_clean_data()
df

,year,month,states-affected-and-category-by-states,category,central-pressure-(mb),max-wind-(kt),name,name_clean,states_list,decade,name_initial,intensity_metric,strong_hurricane,computed_category,computed_numeric_category
0,1851,Jun,"TX, C1",1.0,974,80.0,NaN,nan,[TX],1850,n,0.082136,0,Category 1,1
1,1851,Aug,"FL, NW3; I-GA, 1",3.0,955,100.0,"""Great Middle Florida""",Great Middle Florida,"[FL, GA]",1850,G,0.104712,0,Category 3,3
2,1852,Aug,"AL, 3; MS, 3; LA, 2; FL, SW2, NW1",3.0,961,100.0,"""Great Mobile""",Great Mobile,"[AL, MS, LA, FL]",1850,G,0.104058,0,Category 3,3
3,1852,Sep,"FL, SW1",1.0,982,70.0,NaN,nan,[FL],1850,n,0.071283,0,Category 1,1
4,1852,Oct,"FL, NW2; I-GA, 1",2.0,965,90.0,"""Middle Florida""",Middle Florida,"[FL, GA]",1850,M,0.093264,0,Category 2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,2021,Aug,"LA,4",4.0,931,130.0,Ida,Ida,[LA],2020,I,0.139635,1,Category 4,4
301,2021,Sep,"TX,N1",1.0,991,65.0,Nicholas,Nicholas,[TX],2020,N,0.065590,0,Category 1,1
302,2022,Sep,"FL, SW4; I-FL, SE1;FL, NE1; SC, 1",4.0,941,130.0,Ian,Ian,"[FL, FL, FL, SC]",2020,I,0.138151,1,Category 4,4
303,2022,Nov,&FL - TS,NaN,980,60.0,Nicole,Nicole,"[FL, TS]",2020,N,0.061224,0,Tropical Storm,0


In [16]:
# Extract unique state codes, decades, and computed categories for filters
all_states = sorted(
    {state for sublist in df['states_list'].dropna() for state in sublist}
)
all_decades = sorted(df['decade'].unique())
all_computed_categories = sorted(df['computed_category'].unique())

### 4. Define the App Layout with Filters, Reset Button, and Tabs ###

In [18]:
app.layout = dbc.Container(
    fluid=True,
    style={
        'backgroundColor': '#111111',
        'padding': '20px'
    },
    children=[
        dbc.Row(
            dbc.Col(
                html.H1(
                    'US Hurricanes Analysis Dashboard',
                    className='text-center text-white my-4'),
                width=12
            )
        ),
        # First row: Category, Year Range, and Computed Category filters
        dbc.Row([
            dbc.Col(
                html.Div([
                    dbc.Label(
                        'Select Hurricane Category',
                        className='text-white'
                    ),
                    dcc.Dropdown(
                        id='category-filter',
                        options=[{'label': str(cat), 'value': cat} for cat in sorted(df['category'].dropna().unique())],
                        multi=True,
                        placeholder='Filter by Category',
                        style={'color': 'black'}
                    )
                ], className='mb-3'),
                md=4
            ),
            dbc.Col(
                html.Div([
                    dbc.Label(
                        'Select Year Range',
                        className='text-white'
                    ),
                    dcc.RangeSlider(
                        id='year-slider',
                        min=int(df['year'].min()),
                        max=int(df['year'].max()),
                        value=[int(df['year'].min()), int(df['year'].max())],
                        marks={str(year): str(year) for year in range(int(df['year'].min()), int(df['year'].max())+1, 10)},
                        step=1
                    )
                ], className='mb-3'),
                md=4
            ),
            dbc.Col(
                html.Div([
                    dbc.Label(
                        'Select Computed Category',
                        className='text-white'
                    ),
                    dcc.Dropdown(
                        id='computed-category-filter',
                        options=[{'label': cat, 'value': cat} for cat in all_computed_categories],
                        multi=True,
                        placeholder='Filter by Computed Category',
                        style={'color': 'black'}
                    )
                ], className='mb-3'),
                md=4
            )
        ], className='mb-4'),
        # Second row: Decade and State filters
        dbc.Row([
            dbc.Col(
                html.Div([
                    dbc.Label(
                        'Select Decade(s)',
                        className='text-white'
                    ),
                    dcc.Dropdown(
                        id='decade-filter',
                        options=[{'label': str(dec), 'value': dec} for dec in all_decades],
                        multi=True,
                        placeholder='Filter by Decade',
                        style={'color': 'black'}
                    )
                ], className='mb-3'),
                md=6
            ),
            dbc.Col(
                html.Div([
                    dbc.Label(
                        'Select State(s)',
                        className='text-white'
                    ),
                    dcc.Dropdown(
                        id='state-filter',
                        options=[{'label': state, 'value': state} for state in all_states],
                        multi=True,
                        placeholder='Filter by State',
                        style={'color': 'black'}
                    )
                ], className='mb-3'),
                md=6
            )
        ], className='mb-4'),
        # Third row: Stacked Bar Chart Display Option for Top-N letters
        dbc.Row([
            dbc.Col(
                html.Div([
                    dbc.Label(
                        'Display Top N Letters (leave empty for all)',
                        className='text-white'),
                    dcc.Input(
                        id='top-n-input',
                        type='number',
                        placeholder='e.g., 10',
                        min=1,
                        step=1
                    )
                ], className='mb-3'),
                md=4
            )
        ], className='mb-4'),
        # Reset Filters Button
        dbc.Row(
            dbc.Col(
                dbc.Button(
                    'Reset Filters',
                    id='reset-filters',
                    color='secondary',
                    className='mb-4'
                ),
                width={
                    'size': 2,
                    'offset': 5
                }
            )
        ),
        # Tabs for visualizations and analysis text
        dcc.Tabs([
            dcc.Tab(label="Overview",style={'color': 'black'}, children=[
                dbc.Container([
                    dbc.Row([
                        dbc.Col(
                            dcc.Graph(id='scatter-plot'),
                            md=12)
                    ]),
                    dbc.Row([
                        dbc.Col(
                            dcc.Graph(id='strong-line-plot'),
                            md=12)
                    ]),
                    dbc.Row([
                        dbc.Col(
                            html.Div(
                                id='analysis-text',
                                style={
                                    'color': 'white',
                                    'whiteSpace': 'pre-wrap',
                                    'padding': '15px'
                                }
                            ),
                            md=12
                        )
                    ]),
                    dbc.Row([
                        dbc.Col(
                            html.Div(
                                id='intensity-trend-text',
                                style={
                                    'color': 'white',
                                    'whiteSpace': 'pre-wrap',
                                    'padding': '15px'
                                }
                            ),
                            md=12)
                    ])
                ])
            ]),
            dcc.Tab(label='Geography', style={'color': 'black'}, children=[
                dbc.Container([
                    dbc.Row(
                        dbc.Col(
                            dcc.Graph(id='choropleth-map'),
                            md=12
                        )
                    )
                ])
            ]),
            dcc.Tab(label='Categories', style={'color': 'black'}, children=[
                dbc.Container([
                    dbc.Row([
                        dbc.Col(dcc.Graph(id='category-bar-chart'), md=6),
                        dbc.Col(dcc.Graph(id='box-plot'), md=6)
                    ])
                ])
            ]),
            dcc.Tab(label="Naming Trends", style={'color': 'black'}, children=[
                dbc.Container([
                    dbc.Row(
                        dbc.Col(
                            dcc.Graph(id='stacked-bar-chart'),
                            md=12
                        )
                    ),
                    dbc.Row(
                        dbc.Col(
                            html.Div(
                                id='naming-trend-text',
                                className='text-white p-3'
                            ),
                            md=12)
                    )
                ])
            ]),
            dcc.Tab(label='Distributions', style={'color': 'black'}, children=[
                dbc.Container([
                    dbc.Row(
                        dbc.Col(
                            dcc.Graph(id='histogram'),
                            md=12
                        )
                    )
                ])
            ])
        ])
    ]
)

### 5. Callback to Update Graphs and Analysis Text Dynamically Based on Filters ###

In [20]:
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('strong-line-plot', 'figure'),
     Output('choropleth-map', 'figure'),
     Output('category-bar-chart', 'figure'),
     Output('box-plot', 'figure'),
     Output('histogram', 'figure'),
     Output('stacked-bar-chart', 'figure'),
     Output('analysis-text', 'children'),
     Output('naming-trend-text', 'children'),
     Output('intensity-trend-text', 'children')],
    [Input('category-filter', 'value'),
     Input('year-slider', 'value'),
     Input('computed-category-filter', 'value'),
     Input('decade-filter', 'value'),
     Input('state-filter', 'value'),
     Input('top-n-input', 'value')]
)

def update_graphs(selected_categories, selected_years, selected_computed_categories,
                  selected_decades, selected_states, top_n):
    filtered_df = df.copy()
    filtered_df = filtered_df[(filtered_df['year'] >= selected_years[0]) & 
                              (filtered_df['year'] <= selected_years[1])]
    if selected_decades and len(selected_decades) > 0:
        filtered_df = filtered_df[filtered_df['decade'].isin(selected_decades)]
    if selected_categories and len(selected_categories) > 0:
        filtered_df = filtered_df[filtered_df['category'].isin(selected_categories)]
    if selected_computed_categories and len(selected_computed_categories) > 0:
        filtered_df = filtered_df[filtered_df['computed_category'].isin(selected_computed_categories)]
    if selected_states and len(selected_states) > 0:
        filtered_df = filtered_df[filtered_df['states_list'].apply(
            lambda x: any(s in selected_states for s in x) if isinstance(x, list) else False
        )]
    
    if filtered_df.empty:
        empty_fig = go.Figure()
        empty_fig.update_layout(
            template='plotly_dark',
            title='No Data Available'
        )
        message = 'No data available for the selected filters.'
        return (empty_fig, empty_fig, empty_fig, empty_fig, empty_fig, empty_fig, empty_fig,
                message, message, message)
    
    scatter_fig = create_scatter_plot(filtered_df)
    strong_line_fig = create_strong_hurricanes_line_plot(filtered_df)
    choropleth_fig = create_choropleth_map(filtered_df)
    category_bar_fig = create_category_bar_chart(filtered_df)
    box_fig = create_box_plot(filtered_df)
    hist_fig = create_histogram(filtered_df)
    stacked_bar_fig = create_stacked_bar_chart(filtered_df, top_n=top_n)
    
    strong_df = filtered_df[filtered_df['strong_hurricane'] == 1]
    trend_df = strong_df.groupby('decade', as_index=False)['strong_hurricane'].sum()
    trend_df.rename(columns={'strong_hurricane': 'count'}, inplace=True)
    if len(trend_df) >= 2:
        slope, intercept, r_value, p_value, std_err = linregress(trend_df['decade'], trend_df['count'])
        trend_text = f"Linear Regression: slope = {slope:.3f}, p-value = {p_value:.3f}."
    else:
        trend_text = 'Not enough data for strong hurricane trend analysis.'
    
    intensity_df = filtered_df.groupby('decade')['computed_numeric_category'].mean().reset_index()
    intensity_df.rename(columns={'computed_numeric_category': 'avg_intensity'}, inplace=True)
    intensity_text = "Average computed numeric category per decade:\n" + "\n".join(
        [f"{row['decade']}: {row['avg_intensity']:.2f}" for index, row in intensity_df.iterrows()]
    )
    
    naming_df = filtered_df.groupby(['decade', 'name_initial']).size().reset_index(name='count')
    smoothing_text = ""
    initials = naming_df['name_initial'].unique()
    for initial in initials:
        sub_df = naming_df[naming_df['name_initial'] == initial].sort_values('decade')
        sub_df['ewm'] = sub_df['count'].ewm(span=2, adjust=False).mean()
        smoothing_text += f"Initial {initial}: smoothed counts: " + ", ".join([f"{int(val)}" for val in sub_df['ewm']]) + "\n"
    naming_trend_text = f"Naming Trends Analysis (Exponential Smoothing):\n{smoothing_text}"
    
    analysis_text = f"Trend Analysis on Strong Hurricanes (Category 4 & 5): {trend_text}"
    
    # Wrap output messages in html.Div with whiteSpace set to pre-wrap and text color black
    analysis_div = html.Div(
        analysis_text,
        style={
            'whiteSpace': 'pre-wrap',
            'color': 'white'
        }
    )
    naming_trend_div = html.Div(
        naming_trend_text,
        style={
            'whiteSpace': 'pre-wrap',
            'color': 'white'
        }
    )
    intensity_trend_div = html.Div(
        intensity_text,
        style={
            'whiteSpace': 'pre-wrap',
            'color': 'white'
        }
    )
    
    return scatter_fig, strong_line_fig, choropleth_fig, category_bar_fig, box_fig, hist_fig, stacked_bar_fig, analysis_div, naming_trend_div, intensity_trend_div

### 6. Callback to Reset Filter Values When Reset Button is Clicked ###

In [22]:
@app.callback(
    [Output('category-filter', 'value'),
     Output('year-slider', 'value'),
     Output('computed-category-filter', 'value'),
     Output('decade-filter', 'value'),
     Output('state-filter', 'value'),
     Output('top-n-input', 'value')],
    [Input('reset-filters', 'n_clicks')]
)

def reset_filters(n_clicks):
    """
    Resets all filter components to their default values when the reset button is clicked.
    """
    if n_clicks:
        default_years = [
            int(df['year'].min()),
            int(df['year'].max())
        ]
        return None, default_years, None, None, None, None

    return no_update, no_update, no_update, no_update, no_update, no_update

### 7. Run the App ###

In [24]:
if __name__ == '__main__':
    app.run_server(debug=True)

Interpretation of the Chart and Metrics
- Strong Hurricanes Trend (Category 4 & 5): The linear regression slope is near zero (`slope = 0.005`), and its p-value (`0.436`) indicates no statistically significant upward or downward trend in these high-category storms over time. The polynomial line shows some curvature, but overall there is no strong evidence of a consistent increase or decrease in Category 4 & 5 hurricanes.
- Average Computed Numeric Category: This metric (0 for tropical storms, 1–5 for hurricane categories) generally hovers around 1.8–2.3 for most decades, suggesting the average hurricane intensity is around Category 2. A notable outlier is the 1970s decade, which shows a higher average of 4.00—implying that in the 1970s, the recorded storms had significantly higher wind speeds on average compared to other decades. Otherwise, most decades fall near Category 2, with some variation but no clear long-term escalation.